# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import re
import pandas as pd
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from nltk.corpus import stopwords

nltk.download(['stopwords','wordnet','punkt','omw-1.4','averaged_perceptron_tagger'])

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wxzhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/wxzhao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/wxzhao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/wxzhao/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/wxzhao/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
!pip freeze > requirement.txt

In [4]:
# load data from database
engine = create_engine('sqlite:///../data/database_disaster_response.db')
df = pd.read_sql_table('msg_data.db', con = engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [224]:
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)
predicted_colnames = list(Y.columns)
print(predicted_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [220]:
def tokenize(text):
    # normalizing text to lower case and remove punctuations
    normalize_text = re.sub(r"[^a-zA-Z0-9]+", " ", text.lower())
    words = word_tokenize(normalize_text)

    # remove stop words and lemmatize 
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]

    return tokens

In [109]:
print(X[3])
tokenize(X[3])

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.


['un',
 'report',
 'leogane',
 '80',
 '90',
 'destroyed',
 'hospital',
 'st',
 'croix',
 'functioning',
 'need',
 'supply',
 'desperately']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [221]:
## using the best params found by GridSearch in part below
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('cls',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 123)
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [136]:
Y_pred[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [147]:
print(Y_test.shape)
print(Y_pred.shape)
f1= {}
precision= {}
recall = {}

for col in range(Y_test.shape[1]):
    
    col_name = Y_test.columns[col]
    y_test = Y_test.loc[:, col_name].to_list()
    y_pred = list(Y_pred[:, col])

    report = classification_report(y_test,y_pred, output_dict = True)

    f1[col_name] = report['macro avg']['f1-score']
    precision[col_name] = report['macro avg']['precision']
    recall[col_name] = report['macro avg']['recall']


(8652, 36)
(8652, 36)


/Users/wxzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wxzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wxzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wxzhao/op

In [148]:
print(recall)

{'related': 0.523195048422256, 'request': 0.7386607503374026, 'offer': 0.5, 'aid_related': 0.7729913826833263, 'medical_help': 0.5299106304826985, 'medical_products': 0.529727629088943, 'search_and_rescue': 0.5244857200770137, 'security': 0.4999411764705882, 'military': 0.5170031927456155, 'child_alone': 1.0, 'water': 0.6774746210936389, 'food': 0.8058016268992748, 'shelter': 0.6795422085032862, 'clothing': 0.5346490617726589, 'money': 0.5177664974619289, 'missing_people': 0.5, 'refugees': 0.5106545628284759, 'death': 0.574087750092759, 'other_aid': 0.5184701965392503, 'infrastructure_related': 0.4998154527559055, 'transport': 0.5457454587821917, 'buildings': 0.5717749134757042, 'electricity': 0.521875, 'tools': 0.5, 'hospitals': 0.4999415819605094, 'shops': 0.5, 'aid_centers': 0.5, 'other_infrastructure': 0.49981903727832067, 'weather_related': 0.8244124099969086, 'floods': 0.7350285689361887, 'storm': 0.7503479322103759, 'fire': 0.5052047262013863, 'earthquake': 0.8901268778831211, '

### 6. Improve your model
Use grid search to find better parameters. 

In [149]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7faa33efe310>)),
  ('tfidf', TfidfTransformer()),
  ('cls', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7faa33efe310>),
 'tfidf': TfidfTransformer(),
 'cls': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [152]:
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    'tfidf__sublinear_tf':[False, True],
    'cls__estimator__min_impurity_decrease': [0.0, 0.5]
}

cv = GridSearchCV(pipeline, param_grid = parameters)

In [155]:
cv.fit(X_train,Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7faa33efe310>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('cls',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'cls__estimator__min_impurity_decrease': [0.0, 0.5],
                         'tfidf__sublinear_tf': [False, True],
                         'vect__ngram_range': [(1, 1), (1, 2), (1, 3)]})

In [156]:
Y_pred=cv.predict(X_test)
cv.best_params_

{'cls__estimator__min_impurity_decrease': 0.0,
 'tfidf__sublinear_tf': True,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [237]:
f1= []
precision= []
recall = []

for col in range(Y_test.shape[1]):
    
    col_name = Y_test.columns[col]
    y_test = Y_test.loc[:, col_name].to_list()
    y_pred = list(Y_pred[:, col])

    report = classification_report(y_test,y_pred, output_dict = True)

    f1.append(report['macro avg']['f1-score'])
    precision.append(report['macro avg']['precision'])
    recall.append(report['macro avg']['recall'])
    # f1[col_name] = report['macro avg']['f1-score']
    # precision[col_name] = report['macro avg']['precision']
    # recall[col_name] = report['macro avg']['recall']

results_dict = {'f1':f1, 'precision': precision, 'recall': recall}
results_df = pd.DataFrame(results_dict, index = predicted_colnames)

results_df
# print("RF: f1 - {}, precision - {}, recall - {}".format(
#     sum(f1)/len(f1),
#     sum(precision)/len(precision),
#     sum(recall)/len(recall)))


/Users/wxzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wxzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wxzhao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wxzhao/op

,f1,precision,recall
related,0.567088,0.787411,0.516932
request,0.783358,0.871590,0.739353
offer,0.498929,0.497862,0.500000
aid_related,0.770526,0.784827,0.765008
medical_help,0.519950,0.747958,0.520602
medical_products,0.558393,0.922789,0.537951
search_and_rescue,0.566689,0.863120,0.540003
security,0.495540,0.491215,0.499941
military,0.527890,0.807548,0.518805
child_alone,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [197]:
pipeline_NB = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('cls',MultiOutputClassifier(MultinomialNB()))
])

In [199]:
pipeline_NB.fit(X_train, Y_train)
Y_pred_NB = pipeline_NB.predict(X_test)

In [ ]:
f1_NB= []
precision_NB= []
recall_NB = []

for col in range(Y_test.shape[1]):
    
    col_name = Y_test.columns[col]
    y_test = Y_test.loc[:, col_name].to_list()
    y_pred = list(Y_pred_NB[:, col])

    report = classification_report(y_test,y_pred, output_dict = True)

    f1_NB.append(report['macro avg']['f1-score'])
    precision_NB.append(report['macro avg']['precision'])
    recall_NB.append(report['macro avg']['recall'])
    # f1[col_name] = report['macro avg']['f1-score']
    # precision[col_name] = report['macro avg']['precision']
    # recall[col_name] = report['macro avg']['recall']

print(f1_NB)
print(precision_NB)
print(recall_NB)

In [212]:
print("RF: f1 - {}, precision - {}, recall - {}".format(
    sum(f1)/len(f1),
    sum(precision)/len(precision),
    sum(recall)/len(recall)))

print("NB: f1 - {}, precision - {}, recall - {}".format(
    sum(f1_NB)/len(f1_NB),
    sum(precision_NB)/len(precision_NB),
    sum(recall_NB)/len(recall_NB)))

RF: f1 - 0.607903926985047, precision - 0.8040266109035727, recall - 0.5895031101663097
NB: f1 - 0.515483677916178, precision - 0.5599130886495457, recall - 0.5242322659499304


By compare the results of average f1, precision and recall, RandomForestClassifier overall outperform the Multinominal Naive Bayesian model. Hence, preserve the RF model trained previously.

In [ ]:
## try out adding additional feature
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize, ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('cls',MultiOutputClassifier(RandomForestClassifier()))
])

### 9. Export your model as a pickle file

In [172]:
import pickle

In [176]:
best_model = cv.best_estimator_

In [177]:
with open('RF_multioutput_best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.